In [1]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

In [2]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_A1_WALLET = WalletConfig("A1", "pin")

TRADER_A2_WALLET = WalletConfig("A2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TRADER_A1_WALLET, TRADER_A2_WALLET, TERMINATE_WALLET]

In [3]:
vega = VegaServiceNull(run_with_console=False, use_full_vega_wallet=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/m0/rz86870s4n9fgqh0nzpnxf1w0000gn/T/tmpw8041p86
INFO:vega_sim.null_service:Launching GraphQL node at port 56873
INFO:vega_sim.null_service:Launching Console at port 56879
Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [4]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [5]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

tdai_id = vega.find_asset_id(symbol="tDAI")

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_A1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_A2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

In [6]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=3,
)

market_id = vega.all_markets()[0].id

vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=600,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=1,
    delta_sell=1,
    is_amendment=False,
)

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [7]:
# Make market pass opening auction
vega.submit_order(
    trading_wallet=TRADER_A1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=100,
    wait=True,
)
vega.submit_order(
    trading_wallet=TRADER_A2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=100,
    wait=True,
)

# Post limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=95,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=105,
    wait=True,
)


In [8]:
print("Margin level of Trader 1 (volume=1):", "\n", vega.margin_levels(wallet_name=TRADER_1_WALLET.name, market_id=market_id))
general, margin, bond = vega.party_account(wallet_name=TRADER_1_WALLET.name, asset_id=tdai_id, market_id=market_id)
print("Margin account: ", margin)

Margin level of Trader 1 (volume=1): 
 [MarginLevels(maintenance_margin=5.15315, search_level=5.66846, initial_margin=6.18378, collateral_release_level=7.21441, party_id='47aea5eec9be979a957ae05d3c5a917da7130b4e63365f7a1ae51b08af50db2b', market_id='5024d62e522311404dd7800372c6ed9bbf86ce69e5c22dbefd0e26f36085409e', asset='c71c68741fe73286deb5e8d417e309493075e43cb2b8ddaedf036734def0bc51', timestamp=1638185342259066000)]
Margin account:  6.18378


In [9]:
# Amend order to increase volume
vega.amend_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_id=buy_order_id,
    volume_delta=99,
)

In [10]:
# Check order info if amend successfully
vega.order_status(order_id=buy_order_id)

Using function with raw data from data-node VegaService.order_status. Be wary if prices/positions are not converted from int form


id: "2d98dc923362b476b084c697ca180e1f0cc973915c685d2d0853b4467328b00d"
market_id: "5024d62e522311404dd7800372c6ed9bbf86ce69e5c22dbefd0e26f36085409e"
party_id: "47aea5eec9be979a957ae05d3c5a917da7130b4e63365f7a1ae51b08af50db2b"
side: SIDE_BUY
price: "95000"
size: 10000
remaining: 10000
time_in_force: TIME_IN_FORCE_GTC
type: TYPE_LIMIT
created_at: 1638185338259066000
status: STATUS_ACTIVE
reference: "47aea5eec9be979a957ae05d3c5a917da7130b4e63365f7a1ae51b08af50db2b-1924f80d-4437-4d52-8fc5-d2c9a03086d4"
updated_at: 1638185346259066000
version: 2
batch_id: 1

In [11]:
print("Margin level of Trader 1 (volume=100):", "\n", vega.margin_levels(wallet_name=TRADER_1_WALLET.name, market_id=market_id))
general, margin, bond = vega.party_account(wallet_name=TRADER_1_WALLET.name, asset_id=tdai_id, market_id=market_id)
print("Margin account: ", margin)

Margin level of Trader 1 (volume=100): 
 [MarginLevels(maintenance_margin=515.31421, search_level=566.84563, initial_margin=618.37705, collateral_release_level=721.43989, party_id='47aea5eec9be979a957ae05d3c5a917da7130b4e63365f7a1ae51b08af50db2b', market_id='5024d62e522311404dd7800372c6ed9bbf86ce69e5c22dbefd0e26f36085409e', asset='c71c68741fe73286deb5e8d417e309493075e43cb2b8ddaedf036734def0bc51', timestamp=1638185346259066000)]
Margin account:  618.37705
